In [3]:
import pandas as pd
import seaborn as sns
import new_lib as nl
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
import acquire

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# All imports required to explore the data

In [8]:
telco = acquire.get_telco_data(nl.get_db_url)
telco = telco.iloc[:, 1:]
telco

# Aquiring the Telco data and dropping the Unnamed 0 column to give a dataframe containing the dataset

,payment_type_id,contract_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,contract_type,payment_type
0,2,2,1,0002-ORFBO,Female,0,Yes,Yes,9,Yes,...,Yes,Yes,No,Yes,65.60,593.3,No,DSL,One year,Mailed check
1,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.90,542.4,No,DSL,Month-to-month,Mailed check
2,1,1,2,0004-TLHLJ,Male,0,No,No,4,Yes,...,No,No,No,Yes,73.90,280.85,Yes,Fiber optic,Month-to-month,Electronic check
3,1,1,2,0011-IGKFF,Male,1,Yes,No,13,Yes,...,No,Yes,Yes,Yes,98.00,1237.85,Yes,Fiber optic,Month-to-month,Electronic check
4,2,1,2,0013-EXCHZ,Female,1,Yes,No,3,Yes,...,Yes,Yes,No,Yes,83.90,267.4,Yes,Fiber optic,Month-to-month,Mailed check
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2,2,1,9987-LUTYD,Female,0,No,No,13,Yes,...,Yes,No,No,No,55.15,742.9,No,DSL,One year,Mailed check
7039,1,1,2,9992-RRAMN,Male,0,Yes,No,22,Yes,...,No,No,Yes,Yes,85.10,1873.7,Yes,Fiber optic,Month-to-month,Electronic check
7040,2,1,1,9992-UJOEL,Male,0,No,No,2,Yes,...,No,No,No,Yes,50.30,92.75,No,DSL,Month-to-month,Mailed check
7041,2,3,1,9993-LHIEB,Male,0,Yes,Yes,67,Yes,...,Yes,No,Yes,No,67.85,4627.65,No,DSL,Two year,Mailed check


In [9]:
combos = (['payment_type_id', 'contract_type_id', 'internet_service_type_id', 'gender', 
           'senior_citizen', 'partner', 'dependents', 'phone_service', 'multiple_lines', 
          'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
          'paperless_billing'])
for var in combos:
    chi2, p, degf, expected = chi2, p, degf, expected = stats.chi2_contingency(pd.crosstab(telco.churn, telco[var]))
    if p <= .05:
        print(f'Comparing relationship between churn and {var}')
        print(chi2, p)
        print('------------------')
        print('\n')
        
# Using a chi2 test to determine the main drivers behind the churn at Telco 
# Does not include payments yet, have to drop na and make a seperate test below

Comparing relationship between churn and payment_type_id
648.1423274814 3.6823546520097993e-140
------------------


Comparing relationship between churn and contract_type_id
1184.5965720837926 5.863038300673391e-258
------------------


Comparing relationship between churn and internet_service_type_id
732.309589667794 9.571788222840544e-160
------------------


Comparing relationship between churn and senior_citizen
159.42630036838742 1.510066805092378e-36
------------------


Comparing relationship between churn and partner
158.7333820309922 2.1399113440759935e-36
------------------


Comparing relationship between churn and dependents
189.12924940423474 4.9249216612154196e-43
------------------


Comparing relationship between churn and multiple_lines
11.33044148319756 0.0034643829548773
------------------


Comparing relationship between churn and online_security
849.9989679615962 2.6611496351768565e-185
------------------


Comparing relationship between churn and online_backup
60

In [10]:
telco_test = telco.dropna()
telco_test.total_charges = telco_test.total_charges.replace(' ', 0).astype(float)
# Dropping any null values and converting all the remaining numbers into floats
# Makes it easier to use stats to ttest the charges and churn

In [11]:
combos = ['monthly_charges', 'total_charges']
for var in combos:
    churn = telco_test[telco_test.churn == 'Yes'][var]
    not_churned = telco_test[telco_test.churn == 'No'][var]
    t, p = stats.ttest_ind(churn, not_churned, equal_var = False)
    print(f'Comparing relationship between churn and {var}')
    print(t, p)
    print('------------------')
    print('\n')
# T test for the charges because they we are comparing a catagorical variable to a continuous one
# The code will print the results in a similar format to the chi2 test above

Comparing relationship between churn and monthly_charges
18.407526676414673 8.59244933154705e-73
------------------


Comparing relationship between churn and total_charges
-18.70661828817703 5.902580609073978e-75
------------------


